In [27]:
#Collect all the games with stats from 2008
import pandas as pd
import numpy as np
pt = pd.read_csv("../data/B1_FavGames.csv")
KBBP = pd.read_csv("../data/B3_KPBPIBRP.csv").fillna(0)
pt = pt.loc[(pt['Year']>=2008) & (pt['Round'] !='PI')]
pt['Round'] = pt['Round'].astype(int)
pt = pt.iloc[:,0:10]
pt['PFSeed'] = pt['AFSeed']
pt['PFTeam'] = pt['AFTeam']
pt['PUSeed'] = pt['AUSeed']
pt['PUTeam'] = pt['AUTeam']
ptf = pt.merge(KBBP, left_on=['Year','PFTeam'],right_on=['Year','Team'],how='left')
ptb = ptf.merge(KBBP, left_on=['Year','PUTeam'],right_on=['Year','Team'],how='left')


In [28]:
# Build a model to predict Final Scores
ptn = ptb.select_dtypes(exclude=['object'])
expl = ptn.drop(['AFScore','AUScore'],axis=1)
respF = ptn[['AFScore']]
respU = ptn[['AUScore']]
from sklearn.linear_model import ElasticNet
enet_F = ElasticNet(alpha=.2,l1_ratio=.5)
enet_F.fit(expl,respF)
enet_U = ElasticNet(alpha=.2,l1_ratio=.5)
enet_U.fit(expl,respU)


ElasticNet(alpha=0.2)

In [29]:
# Pick a random Year
from random import choice
p_year = choice([i for i in range(2008,2021) if i not in [2020]])
p_year

2015

In [30]:
# Predict Round 1

expl_R1 = expl.loc[(expl['Year'] == p_year) & (expl['Round'] == 1)]

respF_R1 = enet_F.predict(expl_R1)
respU_R1 = enet_U.predict(expl_R1)
fy = pt.loc[(pt['Year']==p_year)].set_index('Game')
for x in range(1,33):
    fy.loc[x,'PFScore'] = np.around(respF_R1[x-1],3)
    fy.loc[x,'PUScore'] = np.around(respU_R1[x-1],3)
    fy.loc[x,'AWSeed'] = np.where(fy.loc[x,'AFScore']>=fy.loc[x,'AUScore'],fy.loc[x,'AFSeed'],fy.loc[x,'AUSeed'])
    fy.loc[x,'AWTeam'] = np.where(fy.loc[x,'AFScore']>=fy.loc[x,'AUScore'],fy.loc[x,'AFTeam'],fy.loc[x,'AUTeam'])
    fy.loc[x,'PWSeed'] = np.where(fy.loc[x,'PFScore']>=fy.loc[x,'PUScore'],fy.loc[x,'PFSeed'],fy.loc[x,'PUSeed'])
    fy.loc[x,'PWTeam'] = np.where(fy.loc[x,'PFScore']>=fy.loc[x,'PUScore'],fy.loc[x,'PFTeam'],fy.loc[x,'PUTeam'])
    fy.loc[x,'ESPN'] = np.where(fy.loc[x,'AWTeam']==fy.loc[x,'PWTeam'],10,0)

In [31]:
# Predict Round 2
rpt = pt.loc[(pt['Year']==p_year) & (pt['Round']==2)].set_index('Game')

for x in range(1,17):
    rpt.loc[x+32,'PFSeed']=min((fy.loc[2*x-1,'PWSeed']),(fy.loc[2*x,'PWSeed']))
    rpt.loc[x+32,'PUSeed']=max((fy.loc[2*x-1,'PWSeed']),(fy.loc[2*x,'PWSeed']))
    rpt.loc[x+32,'PFTeam']=str(np.where((fy.loc[2*x-1,'PWSeed'])<(fy.loc[2*x,'PWSeed']),fy.loc[2*x-1,'PWTeam'],fy.loc[2*x,'PWTeam']))
    rpt.loc[x+32,'PUTeam']=str(np.where((fy.loc[2*x-1,'PWSeed'])>(fy.loc[2*x,'PWSeed']),fy.loc[2*x-1,'PWTeam'],fy.loc[2*x,'PWTeam']))    
bbs = KBBP.loc[(KBBP['Year']==p_year)]
#rpt['PFTeam'] = rpt['PFTeam'].apply(lambda x: str(x))
rptf = rpt.merge(bbs, left_on = 'PFTeam', right_on = 'Team', how = 'left' )
rptb = rptf.merge(bbs, left_on = 'PUTeam', right_on = 'Team', how = 'left' )
rptn = rptb.select_dtypes(exclude=['object'])
expl_R2 = rptn.drop(['AFScore','AUScore','Year_x','Year_y'],axis=1)

expl_R2['Game'] = expl_R2.index

expl_R2.columns = expl.columns
respF_R2 = enet_F.predict(expl_R2)   #This one is too big
respU_R2 = enet_U.predict(expl_R2)
for x in range(33,49):
    fy.loc[x,'PFScore'] = np.around(respF_R2[x-33],3)
    fy.loc[x,'PUScore'] = np.around(respU_R2[x-33],3)
    fy.loc[x,'AWSeed'] = np.where(fy.loc[x,'AFScore']>=fy.loc[x,'AUScore'],fy.loc[x,'AFSeed'],fy.loc[x,'AUSeed'])
    fy.loc[x,'AWTeam'] = np.where(fy.loc[x,'AFScore']>=fy.loc[x,'AUScore'],fy.loc[x,'AFTeam'],fy.loc[x,'AUTeam'])
    fy.loc[x,'PWSeed'] = np.where(fy.loc[x,'PFScore']>=fy.loc[x,'PUScore'],fy.loc[x,'PFSeed'],fy.loc[x,'PUSeed'])
    fy.loc[x,'PWTeam'] = np.where(fy.loc[x,'PFScore']>=fy.loc[x,'PUScore'],fy.loc[x,'PFTeam'],fy.loc[x,'PUTeam'])
    fy.loc[x,'ESPN'] = np.where(fy.loc[x,'AWTeam']==fy.loc[x,'PWTeam'],20,0)

In [32]:
fy.to_csv('../data/tal.csv')